## Python - Frame Evaluation Framework

### 01 - CPython interpreter & Python bytecode
Python built-in module `dis` (diassembler) can be used to output the bytecode for python functions

The bytecode will show:
- The line number of the source code that corresponds to to the byte code
- List of instructions the CPython interpreter will execute
- Instruction pointers (0,2,4,6) indicate the position of each bytecode instruction in the bytecode sequence (to cover jumps)
- Index of local variable

In [37]:
#Foo-style function
def foo(x, y):
    return x + y

import dis
dis.dis(foo)

  3           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_ADD
              6 RETURN_VALUE


We can also compile the function down to Python bytecode
- def compile(source, filename, mode, flags=0, dont_inherit=False, optimize=-1)

In [74]:
source = """
def foo_compile(x=5, y=7):
    return x + y
"""

# filename: '<string>' to denote code string but could be file made
# mode: 'exec', 'eval' 'single'
bytecode = compile(source, '<string>', 'exec')

# Run the bytecode in cpython - this example it defines the function
exec(bytecode)
dis.dis(bytecode)

result = foo_compile(3, 4)
print(result)


  2           0 LOAD_CONST               5 ((5, 7))
              2 LOAD_CONST               2 (<code object foo_compile at 0x7f18dd538660, file "<string>", line 2>)
              4 LOAD_CONST               3 ('foo_compile')
              6 MAKE_FUNCTION            1 (defaults)
              8 STORE_NAME               0 (foo_compile)
             10 LOAD_CONST               4 (None)
             12 RETURN_VALUE

Disassembly of <code object foo_compile at 0x7f18dd538660, file "<string>", line 2>:
  3           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_ADD
              6 RETURN_VALUE
7


In [105]:
import inspect

def foo_compile(x=5, y=7):
    result = x + y
    frame = inspect.currentframe()
    return result, frame

result, pyt_frame = foo_compile(3, 4)

dis.dis(foo_compile)



  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_ADD
              6 STORE_FAST               2 (result)

  5           8 LOAD_GLOBAL              0 (inspect)
             10 LOAD_METHOD              1 (currentframe)
             12 CALL_METHOD              0
             14 STORE_FAST               3 (frame)

  6          16 LOAD_FAST                2 (result)
             18 LOAD_FAST                3 (frame)
             20 BUILD_TUPLE              2
             22 RETURN_VALUE


In [112]:
# PyFrameObject
print(type(pyt_frame))

print("All attributes of frame")
print(frame.__dir__())

print("Code object:", frame.f_code)
print("Local variables:", frame.f_locals)
#print("Global variables:", frame.f_globals)
print("Value of x:", frame.f_locals['x'])
print("Back pointer to the previous frame:", frame.f_back)
print("Current line number in Python code:", frame.f_lineno)f

<class 'frame'>
All attributes of frame
['__repr__', '__getattribute__', '__setattr__', '__delattr__', 'clear', '__sizeof__', 'f_back', 'f_code', 'f_builtins', 'f_globals', 'f_lasti', 'f_trace_lines', 'f_trace_opcodes', 'f_locals', 'f_lineno', 'f_trace', '__doc__', '__hash__', '__str__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__init__', '__new__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__dir__', '__class__']
Code object: <code object foo_compile at 0x7f18dd3bdbe0, file "/tmp/ipykernel_81/2535985930.py", line 3>
Local variables: {'x': 3, 'y': 4, 'result': 7, 'frame': <frame at 0x7ffa640, file '/tmp/ipykernel_81/2535985930.py', line 6, code foo_compile>}
Value of x: 3
Back pointer to the previous frame: <frame at 0x849ff80, file '/tmp/ipykernel_81/2535985930.py', line 8, code <module>>
Current line number in Python code: 6


In [115]:
#PyCodeObject
print(type(frame.f_code))
print(bytecode.__dir__())

print("========================")
print("Actual bytecode as a bytes object")
print(bytecode.co_code)
print("Constants used in the bytecode")
print(bytecode.co_consts)
print("Names of the variables and functions")
print(bytecode.co_names)
print("Names of the local variables")
print(bytecode.co_varnames)
print("Number of args")
print(bytecode.co_argcount)

<class 'code'>
['__repr__', '__hash__', '__getattribute__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__new__', '__sizeof__', 'replace', 'co_argcount', 'co_posonlyargcount', 'co_kwonlyargcount', 'co_nlocals', 'co_stacksize', 'co_flags', 'co_code', 'co_consts', 'co_names', 'co_varnames', 'co_freevars', 'co_cellvars', 'co_filename', 'co_name', 'co_firstlineno', 'co_lnotab', '__doc__', '__str__', '__setattr__', '__delattr__', '__init__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__dir__', '__class__']
Actual bytecode as a bytes object
b'd\x05d\x02d\x03\x84\x01Z\x00d\x04S\x00'
Constants used in the bytecode
(5, 7, <code object foo_compile at 0x7f18dd538660, file "<string>", line 2>, 'foo_compile', None, (5, 7))
Names of the variables and functions
('foo_compile',)
Names of the local variables
()
Number of args
0


### 02 - Frame Evaluation API https://peps.python.org/pep-0523/
Instead of sending bytecode to interpreter it can be passed to any callback which could modify the bytecode and pass to the interpreter - which is what TorchDynamo does :)

### 03 - TorchDynamo
https://pytorch.org/get-started/pytorch-2.0/

Python:
- Foo is a function call
- This function call becomes a python frame object - each entry of a stack trace essentially
- In each PyFrameObject lives a PyCodeObject which says which the files the functions are from, the constants/local variables
- _PyEval_EvalFrameDefault() - Takes a frame and evalutes it with the interpreter
- (31:44)

Dynamo:
- Uses the same foo function call, the same PyFrameObject and PyCodeObject
- It extracts the torch components of the bytecode and produces several outputs
       - It produces the FX Graph
       - It produces a compiled function from a user-defined compiler using the lowered FX graph as input
       - And it transforms the PyCodeObject bytecode and patches back into the Frame
       - Along the way we accumulate guards to avoid excessive recompilation
  
  
![Image](https://pytorch.org/docs/stable/_images/TorchDynamo.png)

### 03a - TorchDynamo - FX Graph & Graph breaks


### 03b - TorchDynamo - Specialization + Guards

### 03c - TorchDynamo - Backends